In [ ]:
import warnings
warnings.filterwarnings('ignore')

from src.data import DataLoaders
from src import utils
import pandas as pd
from src.models.ModelClasses import FireRiskModels, SmokeAlarmModels
from matplotlib import pyplot as plt
from pathlib import Path
from datetime import date, datetime
import numpy as np
#import geopandas as geo
import subprocess

from IPython.display import display, Markdown

Create report with: `jupyter nbconvert --to pdf --output withMarkdownPDF --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags=['remove_cell']  --no-input testScript.ipynb`

# Fire Models

In [ ]:
# grab version of code 
#subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD'])

In [ ]:
master_log_file_path = utils.DATA['logging'] / 'MasterLog.csv'



In [ ]:
if master_log_file_path.exists():
    experiment_num = pd.read_csv( master_log_file_path).shape[0] + 1 
else:
     experiment_num = 1 
experiment_num

In [ ]:

expt_path =  Path.cwd() / 'Data' /  'Logs' / f'Experiment_{experiment_num}'

In [ ]:
# put all task variables here 
ACS_year = 2016
NFIRS_year_predicted =  2019
NFIRS_num_training_years  =  5
NFIRS_year_used_end = NFIRS_year_predicted -1 

In [ ]:
display(Markdown('**The following columns are being kept**'))
ACS_cols_to_use =[ 'did_not_work_past_12_mo',
               'house_pct_live_alone',
               'pct_alt_heat',
               'house_yr_pct_before_1960',
               'inc_pct_poverty',
               'race_pct_black_or_amind']
print(ACS_cols_to_use)

In [ ]:
Code_repo = subprocess.check_output(['git', 'config', '--get', 'remote.origin.url'])
Code_version =  subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD']).strip()

In [ ]:
expt_log =  pd.DataFrame( {'Experiment': experiment_num ,
 'Date': datetime.now(),
 'expt_folder' : expt_path,
 'Code_Repo' :   Code_repo ,                        
 'Code_Version': Code_version ,
 'ACS_Version': '2016',
 'Level': 'block_group',
 'NFIRS_year_used_start': NFIRS_year_used_end - NFIRS_num_training_years,                                        
 'NFIRS_year_used_end': NFIRS_year_used_end ,
 'NFIRS_year_predicted': NFIRS_year_predicted,  
  'Severity_dollar_estimate' : 10000,
  'ACS_cols_used': [ACS_cols_to_use] 
} )

expt_log = expt_log.set_index('Experiment')


In [ ]:
from datetime import datetime

In [ ]:
print(datetime.now())

In [ ]:
if master_log_file_path.exists():
    expt_log.to_csv ( master_log_file_path,mode ='a', header = False )
else:
    expt_log.to_csv ( master_log_file_path, header = True )
    



In [ ]:
ACS =  DataLoaders.ACSData(ACS_year,'block_group')

In [ ]:
NFIRS = DataLoaders.NFIRSData('block_group',ACS.tot_pop)
NFIRS_Sev = DataLoaders.NFIRSData('block_group',ACS.tot_pop,sev=True)

In [ ]:
NFIRS.fires

In [ ]:


FSM =  FireRiskModels('severity')
FPM =  FireRiskModels('propensity')

## Fire Propensity Model

In [ ]:
FPM.train( NFIRS = NFIRS,
          ACS= ACS,
          ACS_variables = ACS_cols_to_use,
          prediction_year = NFIRS_year_predicted,
          n_years = NFIRS_num_training_years)

## Fire Severity Model

In [ ]:
FSM.train( NFIRS = NFIRS_Sev,
          ACS= ACS,
          ACS_variables = ACS_cols_to_use,
          prediction_year = NFIRS_year_predicted,
          n_years = NFIRS_num_training_years)

# Smoke Alarm Models

In [ ]:
ARC =  DataLoaders.ARCPData(ACS)

In [ ]:
SVI =  DataLoaders.SVIData(ACS.data)
#SVI.data

In [ ]:
Smoke = SmokeAlarmModels()

In [ ]:
smoke_prediction = Smoke.trainModels(ARC,ACS,SVI, ACS_cols_to_use, utils.DATA['data'])

# Saving File

In [ ]:
out_path = utils.DATA['model-outputs'] / f'experiment_{experiment_num}' 
Path.mkdir(out_path, exist_ok = True)

In [ ]:
out = NFIRS.fires.copy()
out.columns =  'fires_per_1K_pop_' + out.columns
cols = out.columns

In [ ]:
out = ACS.data.merge(out, how = 'left', left_index = True, right_index = True)
out = out[cols]

In [ ]:
# Smoke Alarm model munging 


# add number of surveys recorded at the block level
num_surveys_block = Smoke.models['Block_Group']['num_surveys'].copy()
num_surveys_block.index = num_surveys_block.index.str[2:]


out['recorded_home_visits'] = num_surveys_block


# add the statistical model outputs 
Smoke_predicts = Smoke.models['MultiLevel'][
    [ 'detectors_working_prc','detectors_working_CI','geography']].copy()
Smoke_predicts.index = Smoke_predicts.index.str[2:]
out = out.merge(Smoke_predicts,how = 'left', left_index = True, right_index = True)

## add DL model to outputs

out = out.merge(smoke_prediction['Predictions'].copy(),how = 'left', left_index = True, right_index = True)

In [ ]:
out['smoke_detector_risk_frac'] =  1 - out['detectors_predicted'] / 100
out['smoke_detector_risk_score'] =  out['smoke_detector_risk_frac'].rank(pct=True)
out[['smoke_detector_risk_frac','smoke_detector_risk_score']]

In [ ]:
out.shape[0]

In [ ]:
# add final invidual predictors 

out['SVI_Theme_1'] = SVI.data['RPL_THEME1'].replace(-999.0, np.NaN)

out['fire_propensity_risk_score'] = FPM.prediction_probs[1]
out['fire_severity_risk_score'] = FSM.prediction_probs[1]



In [ ]:

#create fire_risk_score by weighing propensity and severity at a .8 to .2 ratio.
#We want the fire_risk_score to be half of the overall combinedRiskScore so multiple the fire_risk_score by 2.
out['fire_prop_weighted'] = out['fire_propensity_risk_score']*0.8
out['fire_sev_weighted'] = out['fire_severity_risk_score']*0.2
out['fire_risk_score'] = out.loc[:,['fire_prop_weighted', 'fire_sev_weighted']].mean(axis = 1, skipna = True)
out['fire_risk_score'] = out['fire_risk_score']*2


# create ranked score for poverty, smoke detector risk and fire risk to be used as the final outputs of the model 
out['fire_risk_ranked'] = out['fire_risk_score'].rank(pct = True)
out['poverty_risk_ranked'] = ACS.data['inc_pct_poverty'].rank(pct = True)
out['smoke_detector_risk_ranked'] = out['smoke_detector_risk_score'].rank(pct = True)

out['CombinedRiskScore'] =  out.loc[:,['smoke_detector_risk_ranked',
                                       'poverty_risk_ranked',
                                       'fire_risk_ranked',
                                       ] ].mean(axis = 1, skipna = True) #False )

In [ ]:
#out

**Combined Risk Score Summary:**

In [ ]:
out['CombinedRiskScore'].describe()

In [ ]:
out_name = f"CombinedModels-basic{date.today()}.csv"
out_save = out.copy()
out_save.index = '#_' + out_save.index
out_save.to_csv(out_path / out_name)

In [ ]:
out_columns = ['state','NFIRS_start','NFIRS_end','years_fires_reported','total_fires_reported', 'max_fires_1k', 
               'median_fires_1k','total_fires_reported_sev', 'max_fires_1k_sev', 'median_fires_1k_sev',
               'fire_propensity_risk_score','fire_severity_risk_score', 'recorded_home_visits',
              'smoke_detector_risk_frac', 'smoke_detector_risk_score','geography', 'SVI_Theme_1','CombinedRiskScore',
              'did_not_work_past_12_mo', 'house_pct_live_alone', 'pct_alt_heat', 'house_yr_pct_before_1960', 
               'inc_pct_poverty', 'race_pct_black_or_amind', 'tot_population', 'house_tot_occ_cnt', 'inc_pcincome', 
               'inc_hhmean', 'race_pct_white', 'race_pct_whitenh','race_pct_whiteh', 'race_pct_nwhitenh', 
               'race_pct_hisp',  'race_pct_black', 'race_pct_amind',  'race_pct_asian', 'race_pct_napi', 'race_pct_other', 
               'race_pct_two+','age_pct_under25', 'age_pct_over65',  'disability_pct', 'snap_past_12_mo', 'single_unit_home', 
               'multi_unit_home', 'mobile_home', 'boat_van_rv_home', 'spanish_speaking', 'state_id', 'state_rank', 'county_id', 'county_rank']
display(Markdown('**Saving columns:**'))
out_columns

In [ ]:
years = [str(num) for num in range(NFIRS_year_used_end - NFIRS_num_training_years,NFIRS_year_used_end+1)  ]

In [ ]:
out['NFIRS_start'] = min(years)
out['NFIRS_start'] = out['NFIRS_start'].astype(int)
out['NFIRS_end'] = max(years)
out['NFIRS_end'] = out['NFIRS_end'].astype(int)
#out

In [ ]:
fires_trained = NFIRS.fires_noAdjustment[years]
fires_trained2 = fires_trained>0
fires_trained2['years_fires_reported']= fires_trained2.sum(axis = 1)
fires_trained2 = fires_trained2['years_fires_reported']
out = out.merge(fires_trained2, how = 'left', left_index = True, right_index = True)

fires_trained = NFIRS.fires_noAdjustment[years]
fires_trained['total_fires_reported'] = fires_trained.sum(axis = 1)
fires_trained = fires_trained['total_fires_reported']
out = out.merge(fires_trained, how = 'left', left_index = True, right_index = True)

fires_trained = NFIRS.fires[years]
fires_trained2 = np.nanmax(fires_trained, axis =1)
fires_trained['max_fires_1k'] = fires_trained2
fires_trained2 = fires_trained['max_fires_1k']
out = out.merge(fires_trained2, how = 'left', left_index = True, right_index = True)

fires_trained = NFIRS.fires[years]
fires_trained2 = np.nanmedian(fires_trained, axis =1)
fires_trained['median_fires_1k'] = fires_trained2
fires_trained2 = fires_trained['median_fires_1k']
out = out.merge(fires_trained2, how = 'left', left_index = True, right_index = True)
#out

In [ ]:
fires_trained = NFIRS_Sev.fires[years]
fires_trained['total_fires_reported_sev'] = fires_trained.sum(axis = 1)
fires_trained = fires_trained['total_fires_reported_sev']
out = out.merge(fires_trained, how = 'left', left_index = True, right_index = True)

fires_trained = NFIRS_Sev.fires[years]
fires_trained2 = np.nanmax(fires_trained, axis =1)
fires_trained['max_fires_1k_sev'] = fires_trained2
fires_trained2 = fires_trained['max_fires_1k_sev']
out = out.merge(fires_trained2, how = 'left', left_index = True, right_index = True)

fires_trained = NFIRS_Sev.fires[years]
fires_trained2 = np.nanmedian(fires_trained, axis =1)
fires_trained['median_fires_1k_sev'] = fires_trained2
fires_trained2 = fires_trained['median_fires_1k_sev']
out = out.merge(fires_trained2, how = 'left', left_index = True, right_index = True)
#out

In [ ]:
ACS_toCombine = ACS.data
#check this makes sense
ACS_toCombine['inc_hhmean'] = ACS_toCombine['inc_pcincome']*ACS_toCombine['tot_population']/ACS_toCombine['house_tot_occ_cnt']
ACS_toCombine['inc_hhmean'] = ACS_toCombine['inc_hhmean'].round()
ACS_toCombine['race_pct_whiteh'] = ACS_toCombine['race_pct_white']-ACS_toCombine['race_pct_whitenh']
ACS_toCombine['race_pct_nwhitenh'] = 1-ACS_toCombine['race_pct_hisp']-ACS_toCombine['race_pct_whitenh']
ACS_toCombine['race_pct_nwhitenh'] = ACS_toCombine['race_pct_nwhitenh'].round(5)


ACS_keep = ACS_toCombine[['state',
    'did_not_work_past_12_mo', 'house_pct_live_alone', 'pct_alt_heat', 'house_yr_pct_before_1960', 
    'inc_pct_poverty', 'race_pct_black_or_amind', 'tot_population', 'house_tot_occ_cnt', 'inc_pcincome', 
    'inc_hhmean', 'race_pct_white', 'race_pct_whitenh','race_pct_whiteh', 'race_pct_nwhitenh', 
    'race_pct_hisp',  'race_pct_black', 'race_pct_amind',  'race_pct_asian', 'race_pct_napi', 'race_pct_other', 
    'race_pct_two+','age_pct_under25', 'age_pct_over65',  'disability_pct', 'snap_past_12_mo', 'single_unit_home', 
    'multi_unit_home', 'mobile_home', 'boat_van_rv_home', 'spanish_speaking'
    ]]
ACS_keep = ACS_keep.replace(-666666666.0, np.nan)
ACS_keep = ACS_keep.replace(np.inf, np.nan)

In [ ]:
out = out.merge(ACS_keep, how = 'left', left_index = True, right_index = True)
#out

In [ ]:
out['state_id'] = out.index
out['state_id'] = out['state_id'].str[:2]
out['state_rank'] = out.groupby('state_id')['CombinedRiskScore'].rank(ascending=False)
out['county_id'] = out.index
out['county_id'] = out['county_id'].str[:5]
out['county_rank'] = out.groupby('county_id')['CombinedRiskScore'].rank(ascending=False)

In [ ]:
out_final = out[out_columns]
#out_final

In [ ]:
a = out.describe()
a.to_csv(out_path/f"summary-pop0-{date.today()}.csv")

In [ ]:
out_name = f"CombinedModels-FileStructureColumns-pop0-{date.today()}.csv"
out_final.index = '#_' + out_final.index
out_final.to_csv(out_path / out_name)